In [1]:
import numpy as np
import pandas as pd
import env

In [2]:
def get_zillow_data():
    sql_query =  '''
    SELECT * FROM properties_2017
    JOIN predictions_2017 USING (parcelid)
    WHERE transactiondate < '2018'
    AND propertylandusetypeid = 261;
    '''
    
    df = pd.read_sql(sql_query, env.get_db_url('zillow'))
    df = df.drop(columns = 'id')
    return df

In [3]:
df = get_zillow_data()
df

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,logerror,transactiondate
0,14297519,NaN,NaN,NaN,3.5,4.0,None,NaN,3.5,NaN,...,485713.0,1023282.0,2016.0,537569.0,11013.72,None,NaN,6.059063e+13,0.025595,2017-01-01
1,17052889,NaN,NaN,NaN,1.0,2.0,None,NaN,1.0,NaN,...,88000.0,464000.0,2016.0,376000.0,5672.48,None,NaN,6.111001e+13,0.055619,2017-01-01
2,14186244,NaN,NaN,NaN,2.0,3.0,None,NaN,2.0,NaN,...,85289.0,564778.0,2016.0,479489.0,6488.30,None,NaN,6.059022e+13,0.005383,2017-01-01
3,12177905,NaN,NaN,NaN,3.0,4.0,None,8.0,3.0,NaN,...,108918.0,145143.0,2016.0,36225.0,1777.51,None,NaN,6.037300e+13,-0.103410,2017-01-01
4,12095076,1.0,NaN,NaN,3.0,4.0,None,9.0,3.0,NaN,...,276684.0,773303.0,2016.0,496619.0,9516.26,None,NaN,6.037461e+13,-0.001011,2017-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52436,12412492,NaN,NaN,NaN,2.0,4.0,None,6.0,2.0,NaN,...,125466.0,346534.0,2016.0,221068.0,4175.08,None,NaN,6.037555e+13,0.001082,2017-09-19
52437,11000655,NaN,NaN,NaN,2.0,2.0,None,6.0,2.0,NaN,...,70917.0,354621.0,2016.0,283704.0,4478.43,None,NaN,6.037101e+13,0.020615,2017-09-20
52438,17239384,NaN,NaN,NaN,2.0,4.0,None,NaN,2.0,NaN,...,50683.0,67205.0,2016.0,16522.0,1107.48,None,NaN,6.111008e+13,0.013209,2017-09-21
52439,12773139,1.0,NaN,NaN,1.0,3.0,None,4.0,1.0,NaN,...,32797.0,49546.0,2016.0,16749.0,876.43,None,NaN,6.037434e+13,0.037129,2017-09-21


In [ ]:
outlier_cols = [col for col in df.columns if col.endswith('_outliers_upper')]
for col in outlier_cols:
    print(col, ': ')
    subset = df[col][df[col] > 0]
    print(f'Number of Observations Above Upper Bound: {subset.count()}', '\n')
    print(subset.describe())
    print('------', '\n')